In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.1 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [ ]:
from typing import Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch

from sklearn.metrics import f1_score
from torch import nn
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import torch.nn.functional as F
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import scipy
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from torch.utils.data import Dataset, DataLoader

In [ ]:
dataset = pd.read_csv('dataset_all_nlp_features_target_classes_no_naive (1).csv', sep='\t')
dataset = dataset[dataset.question_id != '129_Чтение текста - видео']
dataset = dataset[dataset["verb"] > 0]
dataset = dataset[dataset["raven"] > 0]

In [ ]:
dataset.describe()

,task_id,user_id,age,E,A,C,N,O,AP,AM,...,raven_classes,verb_classes,AM_classes,AP_classes,BM_classes,BP_classes,GM_classes,GP_classes,DM_classes,DP_classes
count,6485.000000,6485.000000,1720.000000,6485.000000,6485.000000,6485.000000,6485.000000,6485.000000,6485.000000,6485.000000,...,6485.000000,6485.000000,6485.000000,6485.000000,6485.000000,6485.000000,6485.000000,6485.000000,6485.000000,6485.000000
mean,6.119044,661.120278,23.915116,3.518159,3.679408,3.693600,2.965445,3.750964,3.846289,2.152444,...,1.695143,2.041789,0.516885,1.239784,0.840247,0.914726,0.707941,1.006631,0.687587,1.330918
std,3.530865,275.405688,6.214304,0.827465,0.681960,0.824671,0.894864,0.657391,0.553789,0.685600,...,1.056128,1.332093,0.726832,0.763339,0.811934,0.761489,0.779602,0.773433,0.828719,0.723051
min,1.000000,124.000000,18.000000,1.130000,1.444444,1.330000,1.000000,1.700000,1.670000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,445.000000,20.000000,2.880000,3.222222,3.000000,2.375000,3.300000,3.555556,1.666667,...,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,6.000000,657.000000,22.000000,3.500000,3.670000,3.777778,3.000000,3.700000,3.888889,2.110000,...,2.000000,2.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
75%,9.000000,868.000000,25.000000,4.250000,4.220000,4.440000,3.625000,4.300000,4.222222,2.555556,...,2.000000,3.000000,1.000000,2.000000,2.000000,2.000000,1.000000,2.000000,1.000000,2.000000
max,12.000000,1167.000000,51.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,4.000000,4.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [ ]:
def get_intelligence_labels(dataset):
    dataset['raven_classes'] = dataset['raven_classes'].astype(int)
    RAVEN_LABELS = list(set(dataset['raven_classes'].unique()))
    RAVEN_SCORE_INDICES = range(0, 5)
    VERB_LABELS = list(set(dataset['verb_classes'].unique()))
    VERB_SCORE_INDICES = range(5, 10)

    INT_LABELS = RAVEN_LABELS + VERB_LABELS
    INDEXEX_CIRCLE = [RAVEN_SCORE_INDICES, VERB_SCORE_INDICES]
    return INDEXEX_CIRCLE, INT_LABELS


In [ ]:
curc_indexes, curc_labels = get_intelligence_labels(dataset)

In [ ]:
curc_labels

[0, 1, 2, 3, 4, 0, 1, 2, 3, 4]

In [ ]:
curc_indexes

[range(0, 5), range(5, 10)]

In [ ]:
class DataPreparation(Dataset):
    
    def __init__(self, tokenizer, data, target_values='circumplex', max_length=None):
        
        self.tokenizer = tokenizer
        self.data = data
        self.t_val = target_values
        
        if max_length == None:
            max_length_counted = data["text"].str.split(' ').str.len().max(axis=0)
            self.max_length = max_length_counted if max_length_counted < 512 else 512
        else:
            self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def tokenize(self, text):

        tokens = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt')

        return tokens
    

    def reorganize_labels(self, row):

        def return_zeros(row_data, classes):
          labels_zeros = np.zeros(len(set(self.data[classes].unique())))
          labels_zeros[row_data[classes]] = 1
          return labels_zeros

        return np.concatenate((return_zeros(row, 'raven_classes'), return_zeros(row, 'verb_classes')))   


    def __getitem__(self, index):
        
        source_text = self.data['text'].iloc[index]
        source = self.tokenize(source_text)

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        
        labels = self.reorganize_labels(self.data.iloc[index])
        

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "labels":  labels
        }

In [ ]:
model_name = 'DeepPavlov/rubert-base-cased-sentence'
tokenizer = AutoTokenizer.from_pretrained(model_name)
# bert = AutoModel.from_pretrained(model_name)

In [ ]:
np.random.seed(112)
train_data, vaild_data, test_data = np.split(dataset.sample(frac=1, random_state=42), 
                                     [int(.80*len(dataset)), int(.91*len(dataset))])

print(len(train_data),len(vaild_data), len(test_data))

5188 713 584


In [ ]:
train_data.head(2)

,text,f,filename,task_id,question_id,user_id,gender,age,E,A,...,raven_classes,verb_classes,AM_classes,AP_classes,BM_classes,BP_classes,GM_classes,GP_classes,DM_classes,DP_classes
3168,номер семь,NaN,аудио/answer1_123_19277_6_720.wav,1,140_Рассуждение по заданиям - видео,720,NaN,NaN,4.375,3.555556,...,0,0,1,2,2,1,2,2,1,2
5595,К большому сожалению учиться мне было особо не...,NaN,аудио/answer7_113_13773_1_299.wav,7,134_Видео-интервью,299,NaN,NaN,4.500,3.777778,...,1,1,2,0,0,1,0,1,2,0


In [ ]:
train_dataset_data = DataPreparation(
    tokenizer=tokenizer,
    data = train_data,
    max_length = 120,
)

val_dataset_data = DataPreparation(
    tokenizer=tokenizer,
    data = vaild_data,
    max_length = 120,
)

test_dataset_data = DataPreparation(
    tokenizer=tokenizer,
    data = test_data,
    max_length = 120,
)

In [ ]:
train_dataset = DataLoader(train_dataset_data, batch_size=16, drop_last=True, shuffle=True)
val_dataset = DataLoader(val_dataset_data, batch_size=16)
test_dataset = DataLoader(test_dataset_data, batch_size=16)

In [ ]:
b = next(iter(train_dataset))
for k, v in b.items():
    print(f'{k} shape: {v.shape}')
    # print(b['labels'])

source_ids shape: torch.Size([16, 120])
source_mask shape: torch.Size([16, 120])
labels shape: torch.Size([16, 10])


In [ ]:
class BertBaseline(nn.Module):
    
    def __init__(self, model_name, inner_feautes, out_features):
        super(BertBaseline, self).__init__()

        self.bert = AutoModel.from_pretrained(model_name, return_dict=True)

        self.linear_modules = nn.ModuleList([torch.nn.Linear(self.bert.config.hidden_size, inner_feautes),
                                          torch.nn.Linear(inner_feautes, inner_feautes),
                                          torch.nn.Linear(inner_feautes, out_features)])
        
        self.dropout = nn.Dropout(0.4)
        self.layer_norm = nn.LayerNorm(inner_feautes)
        self.relu = nn.ReLU()   
    
    def forward(self, input_ids, attention_mask):

         bert = self.bert(
            input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True
            )
         x = bert[0][:, 0]  # last hidden state output

         for lin in self.linear_modules[:-1]:
            x = lin(x)
            # x = self.layer_norm(x)
            x = self.relu(x)
            x = self.dropout(x)
        
        #  x = self.relu(x)
         x = self.linear_modules[-1](x)
        
         return x

In [ ]:
model = BertBaseline(model_name, inner_feautes=512, out_features=len(curc_labels))

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-6)
criterion = nn.CrossEntropyLoss()
epochs = 15
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

Надо сделать прогон на каждом классе отдельно, чтобы определеить, какие будут веса можно будет использовать для общего лосса

In [ ]:
def analysing_logits(logits, class_indexes):
  
  def compute_maxes(logits, CUR_SCORE_INDEX):
    labels_zeros = np.zeros(logits[:, CUR_SCORE_INDEX].shape)
    pred_class = np.argmax(logits[:, CUR_SCORE_INDEX], axis=1)
    labels_zeros[list(range(len(labels_zeros))), pred_class] = 1
    return labels_zeros

  list_of_best_labels = []

  for circ_cl in class_indexes:
    labels_zeros = compute_maxes(logits, circ_cl)
    list_of_best_labels.append(labels_zeros)
  
  return np.concatenate((list_of_best_labels), axis=1)

In [ ]:
def init_adding_weights():
    return (1/1.3, 1/1.3)

In [ ]:
def comupte_custom_loss(labels, logits, criterion, class_indexes):
    

    custom_weights = init_adding_weights()

    custom_loss = 0
    for ind, circ_cl in enumerate(class_indexes):
      custom_loss += custom_weights[ind] * criterion(logits[:, circ_cl],
                                  labels[:, circ_cl])

    return custom_loss

In [ ]:
def show_metrics(labels, predicted, class_indexes):

    labels_names = ['raven', 'verb']
    
    labels = np.vstack(labels)
    predicted = np.vstack(predicted)

    for lab, circ_cl in zip(labels_names, class_indexes):
      
      f1_sc = f1_score(labels[:, circ_cl],
               predicted[:, circ_cl], average="macro")
      accur = accuracy_score(labels[:, circ_cl],
               predicted[:, circ_cl])
      
      
      print(f'Scores for {lab}\n==================\n')
      print('Macro F1 score for {0} - {1}\n'.format(lab, f1_sc))
      print('Accuracy score for {0} - {1}\n'.format(lab, accur))
      print(classification_report(labels[:, circ_cl], predicted[:, circ_cl], zero_division=0))


In [ ]:
def train(model, data_loader, device, optimizer, criterion, n_epoch, class_indexes):

    print('Epoch #{}\n'.format(n_epoch+1))

    train_losses = []
    train_labels = []
    train_predictions = []   

    progress_bar = tqdm(total=int(len(data_loader.dataset)/data_loader.batch_size), 
                        desc='Epoch {}'.format(n_epoch + 1))

    model.train()

    for _, data in enumerate(data_loader, 0):


          input_ids = data["source_ids"].to(device)
          attention_mask = data["source_mask"].to(device)
          labels = data['labels'].to(device)

          optimizer.zero_grad()

          pred = model(input_ids=input_ids, attention_mask=attention_mask)

          loss = comupte_custom_loss(labels, pred, criterion, class_indexes)
              
          loss.backward()
              
          optimizer.step()

          resulting_output = analysing_logits(pred.detach().cpu().numpy(), class_indexes)
          

          train_losses.append(loss.item())
          train_labels.extend(labels.cpu().detach().numpy())
          train_predictions.extend(resulting_output)

          progress_bar.set_postfix(loss=np.mean(train_losses))
          progress_bar.update(1)
    progress_bar.update(1)
    progress_bar.close()
  
    
    print('\n\nMean Loss after epoch #{0} - {1}'.format(str(n_epoch + 1), np.mean(train_losses)))
    print(f'\n Scores after {n_epoch + 1} on train:')
    show_metrics(train_labels, train_predictions, class_indexes)

In [ ]:
def validating(model, data_loader, device, optimizer, criterion, n_epoch, class_indexes):

    val_losses, val_labels, val_predictions = [], [], []

    progress_bar = tqdm(total=int(len(data_loader.dataset)/data_loader.batch_size),
                        desc='Epoch {}'.format(n_epoch + 1))

    model.eval()

    for _, data in enumerate(data_loader, 0):
          
          input_ids = data["source_ids"].to(device)
          attention_mask = data["source_mask"].to(device)
          labels = data['labels'].to(device)

          with torch.no_grad():
              pred = model(input_ids, attention_mask)

          loss = comupte_custom_loss(labels, pred, criterion, class_indexes)
          
          resulting_output = analysing_logits(pred.detach().cpu().numpy(), class_indexes)

          val_losses.append(loss.item())
          val_labels.extend(labels.cpu().detach().numpy())
          val_predictions.extend(resulting_output)

          progress_bar.set_postfix(loss=np.mean(val_losses))
          progress_bar.update(1)

    progress_bar.update(1)
    progress_bar.close()
    
    
    valid_stats.append(
        {
            'Val Loss': np.mean(val_losses)
        }
    )

    print('\n\nMean Loss after epoch #{0} - {1}'.format(str(n_epoch + 1), np.mean(val_losses)))
    print(f'\n Scores after {n_epoch + 1} on validation:')
    show_metrics(val_labels, val_predictions, class_indexes)

    return valid_stats

In [ ]:
import os

In [ ]:
valid_stats = []
best_valid_loss = float('inf')

# for each epoch
for epoch in range(epochs):
    # train

    try:
        train(model, train_dataset, device, optimizer, criterion, epoch, curc_indexes)
          # # validate
        valid_stats = validating(model, val_dataset, device, optimizer, criterion, epoch, curc_indexes )

        if valid_stats[epoch]['Val Loss'] < best_valid_loss:
            print(f"Best validation loss - {valid_stats[epoch]['Val Loss']}")
            best_valid_loss = valid_stats[epoch]['Val Loss']
            name_to_save = f'model_baseline_basic_all_circ'
            if os.path.isfile('results/'+name_to_save+'.pth'):
                    os.remove('results/'+name_to_save+'.pth')
                    torch.save(model.state_dict(), 'results/'+name_to_save+'.pth')
            else:
                    if not os.path.isdir('results'):
                        os.mkdir('results')
                    torch.save(model.state_dict(), 'results/'+name_to_save+'.pth')
    except KeyboardInterrupt:
        break

Epoch #1



Epoch 1:   0%|          | 0/324 [00:00<?, ?it/s]



Mean Loss after epoch #1 - 2.4328414195969854

 Scores after 1 on train:
Scores for raven

Macro F1 score for raven - 0.1717024425216555

Accuracy score for raven - 0.36824845679012347

              precision    recall  f1-score   support

           0       0.22      0.04      0.07       925
           1       0.18      0.18      0.18       989
           2       0.44      0.74      0.55      2251
           3       0.17      0.02      0.03       826
           4       0.05      0.02      0.03       193

   micro avg       0.37      0.37      0.37      5184
   macro avg       0.21      0.20      0.17      5184
weighted avg       0.29      0.37      0.29      5184
 samples avg       0.37      0.37      0.37      5184

Scores for verb

Macro F1 score for verb - 0.1767954918139247

Accuracy score for verb - 0.21315586419753085

              precision    recall  f1-score   support

           0       0.21      0.05      0.08       992
           1       0.16      0.06      0.09       

Epoch 1:   0%|          | 0/44 [00:00<?, ?it/s]



Mean Loss after epoch #1 - 2.384304512642387

 Scores after 1 on validation:
Scores for raven

Macro F1 score for raven - 0.12444444444444444

Accuracy score for raven - 0.45161290322580644

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       118
           1       0.00      0.00      0.00       133
           2       0.45      1.00      0.62       322
           3       0.00      0.00      0.00       103
           4       0.00      0.00      0.00        37

   micro avg       0.45      0.45      0.45       713
   macro avg       0.09      0.20      0.12       713
weighted avg       0.20      0.45      0.28       713
 samples avg       0.45      0.45      0.45       713

Scores for verb

Macro F1 score for verb - 0.11733556938416893

Accuracy score for verb - 0.25105189340813466

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       118
           1       0.00      0.00      0.00 

...



Mean Loss after epoch #14 - 2.246657978265713

 Scores after 14 on validation:
Scores for raven

Macro F1 score for raven - 0.16272253171876216

Accuracy score for raven - 0.45161290322580644

              precision    recall  f1-score   support

           0       0.25      0.14      0.18       118
           1       0.00      0.00      0.00       133
           2       0.47      0.95      0.63       322
           3       0.00      0.00      0.00       103
           4       0.00      0.00      0.00        37

   micro avg       0.45      0.45      0.45       713
   macro avg       0.14      0.22      0.16       713
weighted avg       0.25      0.45      0.32       713
 samples avg       0.45      0.45      0.45       713

Scores for verb

Macro F1 score for verb - 0.25438267169846746

Accuracy score for verb - 0.3169705469845722

              precision    recall  f1-score   support

           0       0.30      0.59      0.40       118
           1       0.00      0.00      0.00

Epoch 15:   0%|          | 0/324 [00:00<?, ?it/s]



Mean Loss after epoch #15 - 2.1398568422916533

 Scores after 15 on train:
Scores for raven

Macro F1 score for raven - 0.19713124291008938

Accuracy score for raven - 0.4581404320987654

              precision    recall  f1-score   support

           0       0.44      0.26      0.33       924
           1       0.27      0.01      0.02       990
           2       0.46      0.94      0.62      2250
           3       0.50      0.01      0.01       827
           4       0.00      0.00      0.00       193

   micro avg       0.46      0.46      0.46      5184
   macro avg       0.33      0.24      0.20      5184
weighted avg       0.41      0.46      0.33      5184
 samples avg       0.46      0.46      0.46      5184

Scores for verb

Macro F1 score for verb - 0.30886968251729063

Accuracy score for verb - 0.36921296296296297

              precision    recall  f1-score   support

           0       0.42      0.70      0.53       994
           1       0.29      0.00      0.01    

Epoch 15:   0%|          | 0/44 [00:00<?, ?it/s]



Mean Loss after epoch #15 - 2.249684327779955

 Scores after 15 on validation:
Scores for raven

Macro F1 score for raven - 0.18948076335211145

Accuracy score for raven - 0.45161290322580644

              precision    recall  f1-score   support

           0       0.30      0.35      0.32       118
           1       0.00      0.00      0.00       133
           2       0.49      0.87      0.63       322
           3       0.00      0.00      0.00       103
           4       0.00      0.00      0.00        37

   micro avg       0.45      0.45      0.45       713
   macro avg       0.16      0.24      0.19       713
weighted avg       0.27      0.45      0.34       713
 samples avg       0.45      0.45      0.45       713

Scores for verb

Macro F1 score for verb - 0.23909268068480047

Accuracy score for verb - 0.31136044880785413

              precision    recall  f1-score   support

           0       0.27      0.64      0.38       118
           1       0.00      0.00      0.0

In [ ]:
def test(data_loader, device, class_indexes):

    val_losses, val_labels, val_predictions = [], [], []
    model.load_state_dict(torch.load('results/model_baseline_basic_all_circ.pth'))
    

    model.eval()

    for _, data in enumerate(data_loader, 0):
          
          input_ids = data["source_ids"].to(device)
          attention_mask = data["source_mask"].to(device)
          labels = data['labels'].to(device)

          with torch.no_grad():
              pred = model(input_ids, attention_mask)

          
          resulting_output = analysing_logits(pred.detach().cpu().numpy(), class_indexes)

          val_labels.extend(labels.cpu().detach().numpy())
          val_predictions.extend(resulting_output)



    show_metrics(val_labels, val_predictions, class_indexes)
    # print('F1 score after epoch #{0} on validation - {1}\n'.format(str(n_epoch + 1), f1_score(val_labels, val_predictions, average='macro'))

In [ ]:
test(train_dataset, device, curc_indexes)

Scores for raven

Macro F1 score for raven - 0.1782615490006887

Accuracy score for raven - 0.45621141975308643

              precision    recall  f1-score   support

           0       0.55      0.18      0.28       925
           1       0.00      0.00      0.00       989
           2       0.45      0.98      0.62      2251
           3       0.00      0.00      0.00       826
           4       0.00      0.00      0.00       193

   micro avg       0.46      0.46      0.46      5184
   macro avg       0.20      0.23      0.18      5184
weighted avg       0.29      0.46      0.32      5184
 samples avg       0.46      0.46      0.46      5184

Scores for verb

Macro F1 score for verb - 0.2941157869968735

Accuracy score for verb - 0.3881172839506173

              precision    recall  f1-score   support

           0       0.43      0.72      0.54       994
           1       0.00      0.00      0.00       770
           2       0.36      0.39      0.38      1296
           3      

In [ ]:
test(val_dataset, device, curc_indexes)

Scores for raven

Macro F1 score for raven - 0.15418306200653242

Accuracy score for raven - 0.45582047685834504

              precision    recall  f1-score   support

           0       0.29      0.09      0.14       118
           1       0.00      0.00      0.00       133
           2       0.47      0.98      0.63       322
           3       0.00      0.00      0.00       103
           4       0.00      0.00      0.00        37

   micro avg       0.46      0.46      0.46       713
   macro avg       0.15      0.21      0.15       713
weighted avg       0.26      0.46      0.31       713
 samples avg       0.46      0.46      0.46       713

Scores for verb

Macro F1 score for verb - 0.2413615299430128

Accuracy score for verb - 0.32398316970546986

              precision    recall  f1-score   support

           0       0.29      0.59      0.39       118
           1       0.00      0.00      0.00       115
           2       0.31      0.30      0.31       187
           3    

In [ ]:
test(test_dataset, device, curc_indexes)

Scores for raven

Macro F1 score for raven - 0.156252324193138

Accuracy score for raven - 0.4400684931506849

              precision    recall  f1-score   support

           0       0.35      0.12      0.17        95
           1       0.00      0.00      0.00       111
           2       0.44      0.95      0.61       258
           3       0.00      0.00      0.00        95
           4       0.00      0.00      0.00        25

   micro avg       0.44      0.44      0.44       584
   macro avg       0.16      0.21      0.16       584
weighted avg       0.25      0.44      0.30       584
 samples avg       0.44      0.44      0.44       584

Scores for verb

Macro F1 score for verb - 0.207212950203781

Accuracy score for verb - 0.2791095890410959

              precision    recall  f1-score   support

           0       0.28      0.52      0.37        95
           1       0.00      0.00      0.00       100
           2       0.25      0.31      0.28       153
           3       0.